# 3. vježba: modeliranje nizova povratnim neuronskim mrežama

In [1]:
from __future__ import print_function, division

import os
import sys
import re
import pdb
import time

import numpy as np
import scipy as sp
import tensorflow as tf

## Zadatak 1: učitavanje podataka i batching

In [82]:
# hardcoded paths - change if necessary
root = 'data'

# this one you need to download from the dataset
full_dataset = 'data/movie_lines.txt'

output_destination = 'data/selected_conversations.txt'
movie_selection = 'data/selected_movies.txt'

# separator used in the original dataset
separator = ' +++$+++ '

# movie ID file
MOVIE_ID = 0

# full conversation dataset file
MOVIE_ID_FULL = 2
# reverse indexing
CHARACTER_NAME = -2
CHARACTER_LINE = -1

# keep just these characters for simplicity (and utf8 breaking)
repl = r'[^A-Za-z0-9()\,!\?\'\`\. ]'


# regex replace
def filter(string):
    return re.sub(repl, '', string)


# from a movie ID string (e.g. M134), output the number (134)
def number_from_id(id):
    return int(id[1:])


# read just movie ID's, rest is for readability
def read_selected(path_to_selected_movies):
    selected_movies = set()

    with open(path_to_selected_movies, 'r') as infile:
        for line in infile:
            parts = line.strip().split(separator)
            selected_movies.add(parts[MOVIE_ID].strip())
    return selected_movies


# select and write to output file
def select_and_write(path_to_full_dataset, path_to_output, selected_movies):
    movies = {}

    with open(path_to_full_dataset, 'r', encoding="ISO-8859-1") as infile:

        for line in infile:

            parts = line.strip().split(separator)

            if parts[MOVIE_ID_FULL].strip() not in selected_movies:
                continue

            # take data and transform to tuple
            ID = parts[MOVIE_ID_FULL]
            char_name = parts[CHARACTER_NAME]
            char_line = parts[CHARACTER_LINE]

            tup = (number_from_id(ID), char_name, char_line)

            # add to map
            if ID not in movies:
                movies[ID] = []
            movies[ID].append(tup)

    with open(path_to_output, 'w') as out:
        for movie in movies:
            # sort by line number
            dialogue = sorted(movies[movie], key=lambda t: t[0])
            for n, name, text in dialogue:
                out.write(filter(name) + ':\n' + filter(text) + '\n\n')


def main():
    # uses hardcoded paths
    selection = os.path.join(root, movie_selection)
    selected_movies = read_selected(selection)

    dataset = os.path.join(root, full_dataset)
    output = os.path.join(root, output_destination)
    select_and_write(dataset, output, selected_movies)

In [77]:
class Dataset:
    def preprocess(self, input_file):
        with open(input_file, "r") as f:
            data = f.read()

        # count and sort most frequent characters
        characters, counts = np.unique(list(data), return_index=True)
        self.sorted_characters = characters[np.argsort(-counts)]
        self.vocab_size = len(self.sorted_characters)

        # self.sorted chars contains just the characters ordered descending by frequency
        self.char2id = dict(zip(self.sorted_characters, range(len(self.sorted_characters)))) 
        # reverse the mapping
        self.id2char = {k:v for v,k in self.char2id.items()}
        # convert the data to ids
        self.x = np.array(list(map(self.char2id.get, data)))

    def encode(self, sequence):
        encoded_sequence = np.array([self.char2id[c] for c in sequence], dtype=np.int32)
        return encoded_sequence

    def decode(self, sequence):
        decoded_sequence = [self.id2char[c] for c in sequence]
        return decoded_sequence
        
    def create_minibatches(self, batch_size, sequence_length):
        self.batch_size = batch_size
        self.sequence_length = sequence_length
        self.batch_index = 0
        
        self.num_batches = int(len(self.x) / (batch_size * sequence_length)) # calculate the number of batches
        self.batches = np.zeros([self.num_batches, self.batch_size, self.sequence_length + 1], dtype=np.int32)  
    
        for batch in range(self.num_batches):
            for index in range(self.batch_size):
                start = batch * sequence_length + index * (self.num_batches * sequence_length)
                end = start + self.sequence_length + 1 
                self.batches[batch, index, :] = self.x[start : end]

    def next_minibatch(self):        
        if self.batch_index == self.num_batches:
            self.batch_index = 0

        batch = self.batches[self.batch_index, :, :]
        self.batch_index += 1
        
        batch_x = batch[:, :-1]
        batch_y = batch[:, 1:]
        
        return self.batch_index == self.num_batches, batch_x, batch_y
    
    def _as_one_hot(self, x, vocab):
        n = len(x)
        Yoh = np.zeros((n, vocab))
        Yoh[np.arange(n), x] = 1
        
        return Yoh
    
    def one_hot(self, batch):
        if batch.ndim == 1:
            return self._as_one_hot(batch, self.vocab_size)
        else:
            return np.array([self._as_one_hot(b, self.vocab_size) for b in batch])

In [78]:
# test 1
dat = Dataset()
dat.preprocess("data/selected_conversations.txt")

dat.decode(dat.encode("Tomorrow"))

['T', 'o', 'm', 'o', 'r', 'r', 'o', 'w']

## Zadatak 2: obična jednoslojna povratna neuronska mreža

In [141]:
class RNN():
    # inicijalizacija parametara
    def __init__(self, hidden_size, sequence_length, vocabulary_size, learning_rate):
        self.hidden_size = hidden_size
        self.sequence_length = sequence_length
        self.vocabulary_size = vocabulary_size
        self.learning_rate = learning_rate

        self.U = np.random.normal(size=[vocabulary_size, hidden_size], scale=1.0 / np.sqrt(hidden_size))  # ... input projection
        self.W = np.random.normal(size=[hidden_size, hidden_size], scale=1.0 / np.sqrt(hidden_size))  # ... hidden-to-hidden projection
        self.b = np.zeros([1, hidden_size])

        self.V = np.random.normal(size=[hidden_size, vocabulary_size], scale=1.0 / np.sqrt(vocabulary_size)) # ... output projection
        self.c = np.zeros([1, vocabulary_size]) # ... output bias

        # memory of past gradients - rolling sum of squares for Adagrad
        self.memory_U = np.zeros_like(self.U)
        self.memory_W = np.zeros_like(self.W)
        self.memory_V = np.zeros_like(self.V)
        self.memory_b = np.zeros_like(self.b)
        self.memory_c = np.zeros_like(self.c)
        
    def rnn_step_forward(self, x, h_prev, U, W, b):
        # A single time step forward of a recurrent neural network with a 
        # hyperbolic tangent nonlinearity.

        # x - input data (minibatch size x input dimension)
        # h_prev - previous hidden state (minibatch size x hidden size)
        # U - input projection matrix (input dimension x hidden size)
        # W - hidden to hidden projection matrix (hidden size x hidden size)
        # b - bias of shape (hidden size x 1)

        h_current, cache = None, None
        h_current = np.tanh(np.dot(h_prev, W) + np.dot(x, U) + b)
        cache = (W, x, h_prev, h_current)

        # return the new hidden state and a tuple of values needed for the backward step

        return h_current, cache


    def rnn_forward(self, x, h0, U, W, b):
        # Full unroll forward of the recurrent neural network with a 
        # hyperbolic tangent nonlinearity

        # x - input data for the whole time-series (minibatch size x sequence_length x input dimension)
        # h0 - initial hidden state (minibatch size x hidden size)
        # U - input projection matrix (input dimension x hidden size)
        # W - hidden to hidden projection matrix (hidden size x hidden size)
        # b - bias of shape (hidden size x 1)
        
        h, cache = None, None
        hs = [h0]
        caches = []
        
        for sequence in range(self.sequence_length):
            data = x[:, sequence, :]
            h_current, cache_current = self.rnn_step_forward(data, hs[-1], U, W, b)
            
            hs.append(h_current)
            caches.append(cache_current)
            
        hs = np.array(hs[1:]).transpose((1, 0, 2))
        
        # return the hidden states for the whole time series (T+1) and a tuple of values needed for the backward step
        
        return hs, caches
    
    def rnn_step_backward(self, grad_next, cache):
        # A single time step backward of a recurrent neural network with a 
        # hyperbolic tangent nonlinearity.

        # grad_next - upstream gradient of the loss with respect to the next hidden state and current output
        # cache - cached information from the forward pass        
        dh_prev, dU, dW, db = None, None, None, None
        W, x, h_prev, h_current = cache
        
        # compute and return gradients with respect to each parameter
        # HINT: you can use the chain rule to compute the derivative of the
        # hyperbolic tangent function and use it to compute the gradient
        # with respect to the remaining parameters
        
        # 65. i 66. slajd formule
        da = grad_next * (1 - h_current**2)
        
        dh_prev = np.dot(da, W.T)
        dU = np.dot(x.T, da)
        dW = np.dot(h_prev.T, da)
        db = np.sum(da, axis=0)

        return dh_prev, dU, dW, db


    def rnn_backward(self, dh, cache):
        # Full unroll forward of the recurrent neural network with a 
        # hyperbolic tangent nonlinearity

        dU = np.zeros_like(self.U)
        dW = np.zeros_like(self.W)
        db = np.zeros_like(self.b)

        # compute and return gradients with respect to each parameter
        # for the whole time series.
        # Why are we not computing the gradient with respect to inputs (x)?
        grads = np.zeros_like(dh[0])
        for dh_t, cache_t in reversed(list(zip(dh, cache))):
            grads, dU_t, dW_t, db_t = self.rnn_step_backward(dh_t + grads, cache_t)
            
            dU += dU_t
            dW += dW_t
            db += db_t
        
        return np.clip(dU, -5, 5), np.clip(dW, -5, 5), np.clip(db, -5, 5)
    
    def output(self, h, V, c):
        # Calculate the output probabilities of the network
        return np.dot(h, V) + c
    
    def softmax(self, x):
        x -= np.max(x)
        logits_exp = np.exp(x)
        return logits_exp / np.sum(logits_exp, axis=1, keepdims=True)

    def output_loss_and_grads(self, h, V, c, y):
        # Calculate the loss of the network for each of the outputs

        # h - hidden states of the network for each timestep. 
        #     the dimensionality of h is (batch size x sequence length x hidden size (the initial state is irrelevant for the output)
        # V - the output projection matrix of dimension hidden size x vocabulary size
        # c - the output bias of dimension vocabulary size x 1
        # y - the true class distribution - a tensor of dimension 
        #     batch_size x sequence_length x vocabulary size - you need to do this conversion prior to
        #     passing the argument. A fast way to create a one-hot vector from
        #     an id could be something like the following code:

        #   y[batch_id][timestep] = np.zeros((vocabulary_size, 1))
        #   y[batch_id][timestep][batch_y[timestep]] = 1

        #     where y might be a list or a dictionary.

        loss, dh, dV, dc = None, None, None, None
        
        loss = 0
        dhs = []
        dV = np.zeros_like(self.V)
        dc = np.zeros_like(self.c)
        
        batch_size = len(h)
        
        # calculate the output (o) - unnormalized log probabilities of classes
        # calculate yhat - softmax of the output
        # calculate the cross-entropy loss
        # calculate the derivative of the cross-entropy softmax loss with respect to the output (o)
        # calculate the gradients with respect to the output parameters V and c
        # calculate the gradients with respect to the hidden layer h
        
        for sequence in range(self.sequence_length):
            y_true = y[:, sequence, :]
            h_t = h[:, sequence, :]
            
            o = self.output(h_t, V, c)
            yhat = self.softmax(o)
            
            loss -= np.sum(np.log(yhat) * y_true) / batch_size
            
            dO = (yhat - y_true) / batch_size
            dV += np.dot(h_t.T, dO)
            dc += np.sum(dO, axis=0)
            
            dhs.append(np.dot(dO, V.T))

        return loss, dhs, dV, dc
    
    # updates parameters
    def update(self, batch_size, dU, dW, db, dV, dc):
        # update memory matrices
        # perform the Adagrad update of parameters
        eps = 1e-7
        for x, dx, mem_x in zip([self.U, self.W, self.b, self.V, self.c], [dU, dW, db, dV, dc], [self.memory_U, self.memory_W, self.memory_b, self.memory_V, self.memory_c]):
            mem_x += np.square(dx)
            x -= self.learning_rate * dx / np.sqrt(mem_x + eps)
            
    def step(self, hs, x, y):
        hs, caches = self.rnn_forward(x, hs, self.U, self.W, self.b)
        loss, dh, dV, dc = self.output_loss_and_grads(hs, self.V, self.c, y)
        dU, dW, db = self.rnn_backward(dh, caches)
        
        self.update(len(x), dU, dW, db, dV, dc)
        
        return loss, hs[:, -1, :]

In [142]:
def run_language_model(dataset, max_epochs, hidden_size=100, sequence_length=30, learning_rate=1e-1, sample_every=100):
    vocabulary_size = len(dataset.sorted_characters)
    rnn = RNN(hidden_size=hidden_size, sequence_length=sequence_length, vocabulary_size=vocabulary_size, learning_rate=learning_rate) # initialize the recurrent network

    current_epoch = 0 
    batch = 0

    h0 = np.zeros((dataset.batch_size, hidden_size))
    average_loss = 0
    
    should_print = False
    while current_epoch < max_epochs: 
        e, x, y = dataset.next_minibatch()
        
        if e: 
            current_epoch += 1
            h0 = np.zeros((dataset.batch_size, hidden_size))
            
            current_batch = batch % dataset.num_batches
            print("Epoch: %06d" % (current_epoch), end="\n")
            print("Average_loss: %.4f; Last batch loss: %.4f" % (average_loss / batch, loss))
            print("=====================================================")
            
            should_print = True

        # One-hot transform the x and y batches
        x_oh = dataset.one_hot(x)
        y_oh = dataset.one_hot(y)

        # Run the recurrent network on the current batch
        # Since we are using windows of a short length of characters,
        # the step function should return the hidden state at the end
        # of the unroll. You should then use that hidden state as the
        # input for the next minibatch. In this way, we artificially
        # preserve context between batches.
        loss, h0 = rnn.step(h0, x_oh, y_oh)
        average_loss += loss

        if batch % sample_every == 0: 
            # run sampling (2.2)
            seed = "HAN:\nIs that good or bad?\n\n"
            sampled_data = sample(rnn, seed, 300, dataset)
            
            if should_print:
                print(''.join(sampled_data))
                print() 
                should_print = False
            
        batch += 1
        
def sample(rnn, seed, n_sample, dataset):
    h0 = np.zeros([1, rnn.hidden_size])
    seed_oh = dataset.one_hot(dataset.encode(seed))
    
    sampled = []
    
    for c_oh in seed_oh:
        h0, _ = rnn.rnn_step_forward(c_oh.reshape([1, -1]), h0, rnn.U, rnn.W, rnn.b)
        sampled.append(np.argmax(c_oh))
        
    for i in range(len(seed), n_sample):
        prev_out = np.array([sampled[-1]])
        in_oh = dataset.one_hot(prev_out)
        h0, _ = rnn.rnn_step_forward(in_oh, h0, rnn.U, rnn.W, rnn.b)
        
        probabilities = rnn.softmax(rnn.output(h0, rnn.V, rnn.c))
        out_char_oh = np.random.choice(range(dataset.vocab_size), p=probabilities.ravel()) 
        sampled.append(out_char_oh)
  
    return dataset.decode(sampled)

In [145]:
dataset = Dataset()
dataset.preprocess("data/selected_conversations.txt")
dataset.create_minibatches(batch_size=5, sequence_length=30)
rnn = run_language_model(dataset, 50, sequence_length=dataset.sequence_length)

Epoch: 000001
Average_loss: 66.6664; Last batch loss: 69.4112
HAN:
Is that good or bad?

BECTORE:
Wplrint thath! Mon


DUELY:
What'r pantiwer sous ten't... Yout sidlyt I'm a bit't be it ke me. No. TPING:
Hut a a buactiniin.

MERE FYou'lr tri, ...

GOKE:
Whe'll, at stalg. Wo conk way as to be bassed ceme sulgiks acke?

MONND:
Hente I. Doow ied andyomedurd be et

Epoch: 000002
Average_loss: 62.4517; Last batch loss: 65.9423
HAN:
Is that good or bad?

DONKERSIN:
I con thinve you wall bean, as Guexshede!

INNIE:
And Dake? HR. Hos?

DONDO:
Whagh be out what it the haof the lot guees beand of se to on you they leake ckagunnd mimesit'y wuy. be tety?

DETE:
The gupsting wertigem, I got him thito, a good. Lidts alle. Your wic

Epoch: 000003
Average_loss: 60.4021; Last batch loss: 65.3751
HAN:
Is that good or bad?

DDE. Y:
Ol no, stole there betran go all coonto! ber bive, ever.  Golyon.

IVERETTh:
Fo alit't it nigh the show if?

DDONTH:
Ary tua cwool on methang. Hlithing weasishote lit a fe  Ch

Epoch: 000021
Average_loss: 52.9073; Last batch loss: 59.6862
HAN:
Is that good or bad?

HAD:
Thele dim. I lesper.

DONNA:
I've mencicw. I jadd anythertaok do I want on was a fuck crick?. neen a a feane octafe.

DN:
Brosanicg ease us one claine you.  Do, wild will.

PINDO:
I could haver?

EDDY:
Well Tilds didn't think right?  Peal you do rogure?

AN:
I lill.



Epoch: 000022
Average_loss: 52.7680; Last batch loss: 59.6035
HAN:
Is that good or bad?

MUDDY:
What with your about mownicus hoily want tilk aln percer my shit wett. This Thred the rebrert!.. don't we're hation. Pliald did He compat of this formapse a geten jadd to lig!

DONNIE:
Frightes of up.

DONNIE:
Than for come brong to rad on is your youp thought at st

Epoch: 000023
Average_loss: 52.6365; Last batch loss: 59.4965
HAN:
Is that good or bad?

HENGUROM:
What comeshonde Tike wilk onland id. It an carkenilak, alo wain'..s like have.

DRITHY:
Ohath themafulyind has packin' nielly  you Use fuse wome flasher.

RATTEN:
En't shoul

Epoch: 000041
Average_loss: 51.1136; Last batch loss: 59.2898
HAN:
Is that good or bad?

46N:
YeO, the baffir! N:
Okt you imont. Is is to and us to damen it, Frow never?

MONZOREFTEMEN:
Aleves dour?

JUND.

MR. NIS.E:
VIt., scuo live now that just sack couch to botul... you plar, Door!

MOADO:
It's to this his look orry bestoges and apotens I was onybosies en 

Epoch: 000042
Average_loss: 51.0568; Last batch loss: 59.3576
HAN:
Is that good or bad?

AOOSTY:
He watting 're and for himbpom She week to.

CINSSE:
Whroe, Frainy, is she Are ampsedles.  Now with do. Wo seann.

LOUE:
Not can''ll dayb.

MIM:
P mupp. That's pother.

COWBOY:
What I woulddycontitn.  Fut ond us being to just younet?

SAID PER:
Dattair, came munts 

Epoch: 000043
Average_loss: 51.0020; Last batch loss: 59.1546
HAN:
Is that good or bad?

EVERETT:
Thtou.

CWOPPRAN:
It's pahen't co retuy, thr.

CONNINERY THURMANLERIS

PETE:
I here. It's lose. We're from soy.

MR. BOY:
No.

VODA:
Has just got it. IVEgot, If here.

DUKN:
